# Getting Started: Hybrid Renewable Energy Forecasting and Trading Competition

This notebook is a quick start guide intended to help participants get going
quickly, to support understanding of the data and tasks, and to provide a 
transparent benchmark method that will freature in the competition leaderboard.


## Requirements
First, let's load some useful packages for working with the energy and weather
data we'll need for the competition tasks.

In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.iolib.smpickle import load_pickle
import comp_utils

import plotly.express as px
import plotly.graph_objects as go

import pickle as pkl

## Weather Data
Weather data is supplied in netCDF format. This is format is designed for
efficient manipulation of data on high-dimensional grids. In our case, the
dimensions are space (latitude and longitude) and time (forecast base time, 
and lead-time).
Weather data from two providers, is made available by the competition, though
you may include others.
A static dataset is provided with three years worth of weather forecasts for
model development and training. Live weather data is available via API to
update this dataset and for generate live forecasts and market bids to submit
as part of the competition.

Let's open the file containing the DWD weather forecasts for the region
containing the Hornsea 1 wind farm. Then take the average wind speed at a
height of 100m, and convert this to a Pandas DataFrame. It is also helpful
to convert the "valid_time" to a time stamp. Then do the same for the solar
radiation forecasts covering PES region 10.

In [ ]:
dwd_Hornsea1 = xr.open_dataset("data/dwd_icon_eu_hornsea_1_20200920_20230920.nc")
dwd_Hornsea1_features = dwd_Hornsea1["WindSpeed:100"].mean(dim=["latitude","longitude"]).to_dataframe().reset_index()
dwd_Hornsea1_features["reference_time"] = dwd_Hornsea1_features["reference_time"].dt.tz_localize("UTC")
dwd_Hornsea1_features["valid_time"] = dwd_Hornsea1_features["reference_time"] + pd.TimedeltaIndex(dwd_Hornsea1_features["valid_time"],unit="hours")

In [ ]:
dwd_solar = xr.open_dataset("data/dwd_icon_eu_pes10_20200920_20230920.nc")
dwd_solar_features = dwd_solar["SolarDownwardRadiation"].mean(dim="point").to_dataframe().reset_index()
dwd_solar_features["reference_time"] = dwd_solar_features["reference_time"].dt.tz_localize("UTC")
dwd_solar_features["valid_time"] = dwd_solar_features["reference_time"] + pd.TimedeltaIndex(dwd_solar_features["valid_time"],unit="hours")

## Energy Data
Now let's load the energy data. The volume of energy generation credited in
the wholesale electricity market is the metered output (in units of MWh) minus the bid and offer
accepted volumes (curtailed energy). We need to be careful with unit here too:
the generation datais in units of MW, so needs to be multiplied by 0.5 to calculate
the amount of energy generated in each 30-minute settlement period.
Bid Offer Acceptaces are already in units of MWh.
We should also convert the solar power production to units of MWh.

In [ ]:
energy_data = pd.read_csv("data/Energy_Data.csv")
energy_data["dtm"] = pd.to_datetime(energy_data["dtm"])
energy_data["Wind_MWh_credit"] = 0.5*energy_data["Wind_MW"] - energy_data["boa"]
energy_data["Solar_MWh_credit"] = 0.5*energy_data["Solar_MW"]
energy_data

In [ ]:
fig = px.line(energy_data, x="dtm", y=["Solar_MWh_credit","Wind_MWh_credit"],
              title='Generation',
              labels=dict(value="Energy [MWh]", dtm="Date/Time [30-minute period]"),
              template="plotly_dark")
fig.show()

## Merge Energy and Weather Data
We can merge the weather and enregy data on their timestamps. Since we are only
interested in forecasting day-ahead, I'll also drop the longer lead-times from
the weather forecasts. A scatter plot of key weather variables agains the wind
and solar generation illustrates the forecasting challenge: given a weather forecast,
how much energy do we expect to generate?

In [ ]:
modelling_table = dwd_Hornsea1_features.merge(dwd_solar_features,how="outer",on=["reference_time","valid_time"])
modelling_table = modelling_table.set_index("valid_time").groupby("reference_time").resample("30T").interpolate("linear")
modelling_table = modelling_table.drop(columns="reference_time",axis=1).reset_index()
modelling_table = modelling_table.merge(energy_data,how="inner",left_on="valid_time",right_on="dtm")
modelling_table = modelling_table[modelling_table["valid_time"] - modelling_table["reference_time"] < np.timedelta64(50,"h")]
modelling_table.rename(columns={"WindSpeed:100":"WindSpeed"},inplace=True)

In [ ]:
px.scatter(modelling_table, x="WindSpeed", y="Wind_MWh_credit",
           labels=dict(WindSpeed="Wind Speed [m/s]", Wind_MWh_credit="Generation [MWh]"),
           template="plotly_dark").show()
px.scatter(modelling_table, x="SolarDownwardRadiation", y="Solar_MWh_credit",
           labels=dict(SolarDownwardRadiation="Solar Radiation Downwards [w/m^2]", Solar_MWh_credit="Generation [MWh]"),
           template="plotly_dark").show()

## Modelling
Now we have joined all of our data we can fit a model to predict the total
energy production (wind + solar) for a given  weather forecast. The forecasting
track of the competition requires prediction of quantiles from 10% to 90% in
10% increments.

In this example, we'll throw away missing data and create a target variable
`total_generation_MWh`, but you could do something else.

In [ ]:
modelling_table = modelling_table[modelling_table["SolarDownwardRadiation"].notnull()]
modelling_table = modelling_table[modelling_table["WindSpeed"].notnull()]
modelling_table["total_generation_MWh"] = modelling_table["Wind_MWh_credit"] + modelling_table["Solar_MWh_credit"]

This example uses splines and quantile regression to predict the `total_generation_MWh`
based on the `WindSpeed` and `SolarDownwardRadiation` features we created earlier. In
the loop below we fit a seperate model for each quantile, and store the model in
a dictionary.

In [ ]:
mod = smf.quantreg('total_generation_MWh ~ bs(SolarDownwardRadiation,df=5) + bs(WindSpeed,df=5)', data=modelling_table)

forecast_models = dict()
for quantile in range(10,100,10):
    forecast_models[f"q{quantile}"] = mod.fit(q=quantile/100,max_iter=2500)
    modelling_table[f"q{quantile}"] = forecast_models[f"q{quantile}"].predict(modelling_table)
    modelling_table.loc[modelling_table[f"q{quantile}"] < 0, f"q{quantile}"] = 0

We'll also save the models to save time later, and to use in automated
competition submissions. (These saved models are loaded and used to submit
forecast and market bids in the `auto_submitter.py` script.)

In [ ]:
for quantile in range(10,100,10):
    forecast_models[f"q{quantile}"].save(f"data/model_q{quantile}.pickle")

Let's plot some of the predicted quantiles to see what our (in-sample) forecasts
look like. (Change `ref_time` to see forecasts produced at different times).

In [ ]:
ref_time = modelling_table["reference_time"] == modelling_table["reference_time"][10]

fig = px.line(modelling_table[ref_time],x="valid_time",y="total_generation_MWh",
              labels=dict(valid_time="Date/Time [30-minute period]",total_generation_MWh="Generation [MWh]"),
               title=f"Forecast refernce time: {modelling_table[ref_time]['reference_time'][0]}",
               template="plotly_dark")

for quantile in range(10,100,10):
    fig.add_trace(go.Scatter(x=modelling_table[ref_time]["valid_time"],
                             y=modelling_table[ref_time][f"q{quantile}"],
                            mode='lines',
                            line=dict(color='grey', width=1),
                            name=f"q{quantile}"))
    
fig.update_yaxes(range=[0, 1600])

fig.show()


## Generate a competition submission

So far we have loaded historic data and fit models to predict the total wind 
and solar power generation. Now we'll look at how to retrieve the latest
weather forecast and submit our forecasts and market bids to the competition
platform.

The steps are as follows:

1. Retrieve the latest weather forecasts via the competition API
2. Calculate the features we need as inputs to our models
3. Generate our forecasts using the saved models
4. Wrangle our forecasts and market bids into the necessary format for submission
5. Submit via API!

These are the steps carried out in `auto_submitter.py`, which illustrates
how the process may be automated (see `README`).

This example is based on energy forecasting only, the market bid we'll submit
will be equal to the 50% quantile forecast. You might like to develop a more
sophisticated trading strategy using additional data and forecast of prices.
The competition API provides access to relevant data from the GB electricity
market, and you may make use of any other data you like as well. 

The `comp_utils` modele includes some wrappers to the API endpoints. We'll
use them to download the data and convert it to `xarray` format to match
the historic data we've used already.

In [7]:
latest_dwd_Hornsea1 = comp_utils.weather_df_to_xr(comp_utils.get_hornsea_dwd())
latest_dwd_solar = comp_utils.weather_df_to_xr(comp_utils.get_pes10_nwp("DWD_ICON-EU"))

200
200


And then calculate our weather features and  merge the wind and solar data as before...

In [12]:
latest_dwd_Hornsea1_features = latest_dwd_Hornsea1["WindSpeed:100"].mean(dim=["latitude","longitude"]).to_dataframe().reset_index()
latest_dwd_solar_features = latest_dwd_solar["SolarDownwardRadiation"].mean(dim="point").to_dataframe().reset_index()

latest_forcast_table = latest_dwd_Hornsea1_features.merge(latest_dwd_solar_features,how="outer",on=["ref_datetime","valid_datetime"])
latest_forcast_table = latest_forcast_table.set_index("valid_datetime").resample("30T").interpolate("linear",limit=5).reset_index()

latest_forcast_table.rename(columns={"WindSpeed:100":"WindSpeed"},inplace=True)

Now we can load our forecast models and add the predictive quantiles of total
wind and solar generation to the `latest_forecast_table`. (We could also use
the models stored in the `forecast_models` dictionary, as in two commented out
lines.)

In [14]:
for quantile in range(10,100,10):
    loaded_model = load_pickle(f"data/model_q{quantile}.pickle")
    latest_forcast_table[f"q{quantile}"] = loaded_model.predict(latest_forcast_table)
    
    # latest_forcast_table[f"q{quantile}"] = forecast_models[f"q{quantile}"].predict(latest_forcast_table)
    # latest_forcast_table.loc[latest_forcast_table[f"q{quantile}"] < 0, f"q{quantile}"] = 0


Let's have a look at a plot of out energy generation forecast based on the
latest weather forecasts to make sure everything is as expected. 

In [19]:
fig = px.line(latest_forcast_table,x="valid_datetime",y="q50",
              labels=dict(valid_datetime="Date/Time [30-minute period]",total_generation_MWh="Generation [MWh]"),
              title=f"Forecast refernce time: {latest_forcast_table['ref_datetime'][0]}",
              template="plotly_dark")

for quantile in range(10,100,10):
    fig.add_trace(go.Scatter(x=latest_forcast_table["valid_datetime"],
                             y=latest_forcast_table[f"q{quantile}"],
                            mode='lines',
                            line=dict(color='grey', width=1),
                            name=f"q{quantile}"))
    
fig.update_yaxes(range=[0, 1600])

fig.show()


c:\Users\jb593c\Anaconda3\envs\comp23\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



Finally, we need to wrangle our forecast into the correct format the submit
to the competition platform. Some tools are provided in the `comp_utils` module
to help, but you MUST be sure you're submission are correct and take full
responsibilty for them.

The day-ahead electricity market in Great Britain is synchronised with the European
market, which runs from midnight to midnight in the CET/CEST timezone. We must submit
bids for each half-hour of this time range.

The function `comp_utils.day_ahead_market_times()` creates a pandas DataFrame
with the correct timestamps, which handle the transition to and from daylight
saving. (March 31 2024 only contains be 23 hours!!!)

As we are using timezone-aware timestamps, we can make sure to submit the correct
forecasts and bids by merging onto the DataFrame created by `comp_utils.day_ahead_market_times()`.
In this example, our market bid will be equal to our 50% quantile forecast.

The function `comp_utils.prep_submission_in_json_format()` converts out `submission_data`
into the correct format (using column names `datetime`,`q10`,...,`q90` and `market_bid`).

The function `comp_utils.submit()` will send our data to the competition platform and
print the response from the API.

In [16]:

submission_data=pd.DataFrame({"datetime":comp_utils.day_ahead_market_times()})
submission_data = submission_data.merge(latest_forcast_table,how="left",left_on="datetime",right_on="valid_datetime")
submission_data["market_bid"] = submission_data["q50"]

submission_data = comp_utils.prep_submission_in_json_format(submission_data)
print(submission_data)

# comp_utils.submit(submission_data)

{'submission': [{'timestamp': '2023-10-21T23:00:00+01:00', 'market_mid': 296.97272846978, 'probabilistic_forecast': {10: 170.37139388872137, 20: 216.20577570871782, 30: 248.10177291946385, 40: 273.57744976922106, 50: 296.97272846978, 60: 319.81787361144825, 70: 344.48232462166743, 80: 376.8913597490357, 90: 427.81227625863806}}, {'timestamp': '2023-10-21T23:30:00+01:00', 'market_mid': 266.3617029870919, 'probabilistic_forecast': {10: 145.51619033093883, 20: 187.63317455999533, 30: 218.224911055143, 40: 243.17604604286717, 50: 266.3617029870919, 60: 289.2894478772945, 70: 314.3451677446801, 80: 348.081351462903, 90: 402.17040619731125}}, {'timestamp': '2023-10-22T00:00:00+01:00', 'market_mid': 236.07179656283674, 'probabilistic_forecast': {10: 121.5853228131087, 20: 159.8973347362049, 30: 188.98519445429397, 40: 213.25661377013571, 50: 236.07179656283674, 60: 258.9144498578823, 70: 284.228153001985, 80: 319.0471555520157, 90: 375.95323049401014}}, {'timestamp': '2023-10-22T00:30:00+01:0